In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import NMF

In [3]:
import glob
import os
from collections import defaultdict
import re
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

In [4]:
import nltk
import os
import spacy
import re
import string
import pickle
from itertools import cycle
import matplotlib.pyplot as plt
from nltk import stem
%matplotlib inline
%config InlineBackend.figure_formats = ['retina']
from nltk.tokenize import word_tokenize, sent_tokenize, MWETokenizer
from gensim import corpora, models, similarities, matutils
from sklearn.decomposition import TruncatedSVD, NMF
from corextopic import corextopic, vis_topic
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import scattertext as st

In [5]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

In [6]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

## Spotify Audio Features

In [ ]:
df_spot = pd.read_csv('../data/df_emo_top_long_nm_spfy.csv')

In [ ]:
df_spot.head(2)

**avg scores per topic model**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def spotfeats(data,alb):
    df_spot2 = pd.DataFrame()
    df_spot2 = data.loc[data['album'] == alb] # ) & (data['topic']==topics)]    
    #df_spot2 = data.loc[data['subgenre'] == alb] # ) & (data['topic']==topics)]    
    #df_spot3 = df_spot2.loc[df_spot2['year']<=2018]
    df_spot2 = df_spot2[['acousticness',   'instrumentalness', 'tempo', 'speechiness', 'valence','energy','liveness','danceability','loudness']]
    
    # get mean of scores
    df_spot2 = df_spot2[['acousticness',   'instrumentalness', 'tempo', 'speechiness', 'valence','energy','liveness','danceability','loudness']].mean()    
    df_spot3 = pd.DataFrame(df_spot2)
    df_spot3_T = df_spot3.T
    df_spot3_T['loudness'] = abs(df_spot3_T['loudness'])
       
    
    return df_spot3_T

### Search baesd - 3 matches on Radiohead

In [ ]:
df_spot_kkoala=spotfeats(df_spot,'Music to Draw to: Satellite')

In [ ]:
df_spot_kkoala.head(10)

**Radar function**

In [ ]:
import base64
from io import StringIO

# import matplotlib.pyplot as plt
# import streamlit as st
# from matplotlib import numpy as np

# def render_svg(svg_file):

#     with open(svg_file, "r") as f:
#         lines = f.readlines()
#         svg = "".join(lines)

#         """Renders the given svg string."""
#         b64 = base64.b64encode(svg.encode("utf-8")).decode("utf-8")
#         html = r'<img src="data:image/svg+xml;base64,%s"/>' % b64
#         return html

In [ ]:
import math

In [ ]:
%matplotlib inline

def radar(data,gen,colors):

    # convert column names into a list
    categories=list(data.columns)
    # number of categories
    N=len(categories)

    # create a list with the average of all features
    value = list(data.mean())
    
    # tempo - scaled
    value[2] = value[2]/220
    
    # speech - scaled
    value[3] = value[3]*1.2
    
    # inst - scaled
    value[1] = value[1] # *2
    
    # loudness - scaled
    value[8] = value[8]/25
    
    # repeat first value to close the circle
    # the plot is a circle, so we need to "complete the loop"
    # and append the start value to the end.
    value+=value[:1]
    
    # calculate angle for each category
    angles=[n/float(N)*2*math.pi for n in range(N)]
    angles+=angles[:1]

    # plot
    fig=plt.figure(figsize = (10,10))

    ax = fig.add_subplot(221, polar=True)

    #plot 1 hits
    ax.plot(angles, value,  linewidth=2, label = gen, color= colors)
    ax.fill(angles, value, alpha=0.35, facecolor=colors)
    ax.set_ylim(0, 1)
    
    ax.grid(True)

    # Add legend
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1),fontsize='large')

    plt.xticks(angles[:-1],categories, size=12,color='black')
    plt.yticks(color='grey',size=12)

    # Create a color palette:
    plt.cm.get_cmap("Set2", len(data.index))
    plt.savefig(f'../img/radar_{gen}.png',transparent=True)
    #plt.savefig(f'../img/radar_{gen}.svg',transparent=True)
    #plt.savefig(f'../img/radar_{gen}.png',transparent=True)
    plt.show()

In [ ]:
radar(df_spot_kkoala,'kkoala','skyblue')

In [ ]:
df_spot_rglasp=spotfeats(df_spot,'ArtScience')

In [ ]:
df_spot_rglasp.head(10)

In [ ]:
radar(df_spot_rglasp,'rglasp','skyblue')

In [ ]:
df_spot_rglasp2=spotfeats(df_spot,'Covered')

In [ ]:
df_spot_rglasp2.head(10)

In [ ]:
radar(df_spot_rglasp2,'rglasp2','skyblue')

### by Genre (electronic , dark ambient , belong)

In [ ]:
df_spot_belong1=spotfeats(df_spot,'Common Era')

In [ ]:
df_spot_belong1.head(10)

In [ ]:
radar(df_spot_belong1,'belong1','skyblue')

In [ ]:
df_spot_belong2=spotfeats(df_spot,'Colorloss Record EP')

In [ ]:
df_spot_belong2.head(10)

In [ ]:
radar(df_spot_belong2,'belong2','skyblue')

In [ ]:
df_spot_belong3=spotfeats(df_spot,'October Language')

In [ ]:
df_spot_belong3.head(10)

In [ ]:
radar(df_spot_belong3,'belong3','skyblue')

In [ ]:
df_spot_alog=spotfeats(df_spot,'Unemployed')

In [ ]:
df_spot_alog.head(10)

In [ ]:
radar(df_spot_alog,'alog','skyblue')

In [ ]:
df_spot_jaga=spotfeats(df_spot,'One-Armed Bandit')

In [ ]:
df_spot_jaga.head(10)

In [ ]:
radar(df_spot_jaga,'jaga','skyblue')

### Get first severl words of review

In [7]:
df_rev = pd.read_csv('../data/df_emo_top_long_nm_spfy.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2014,2015) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
df_rev['revwords'] = df_rev['review']

In [9]:
df_rev.revwords.head(2)

0    Fifty-two tracks deep, this archival compendiu...
1    Fifty-two tracks deep, this archival compendiu...
Name: revwords, dtype: object

In [10]:
words = 'I am confused looking for 3 words from the front'
' '.join(words.split()[:3])

'I am confused'

In [11]:
revwords= df_rev['revwords'].astype(str)

In [17]:
type(revwords)

pandas.core.series.Series

In [18]:
x=revwords.to_string()

In [19]:
y=' '.join(x.split()[:10])

In [20]:
y

'0 Fifty-two tracks deep, this archival compendiu... 1 Fifty-two tracks'

In [21]:
df_rev['revwords'] = df_rev['review'].to_string()

In [22]:
df_rev['revwords10'] = ' '.join(df_rev['revwords'].split()[:10])

AttributeError: 'Series' object has no attribute 'split'